# Generador

Se parte de un dataframe base (generado a partir de los datos meteorológicos) para generar dos tipos de datos fundamentales: la **velocidad de la máquina** y los **valores de color LAB**.


In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

## Variables globales

In [2]:
RUTA_DATOS = r"C:\Users\jaume\Documents\Proyecto\datos"
FIC_ENTRADA="01_Temp_Humedad.pkl"
FIC_TOTAL="02_datos_limpios.pkl"
FIC_SALIDA="02_datos_limpios_sin_parada.pkl"

INTERVALOS_TIEMPO_POR_DIA = 500
FECHA_INICIO="2023-01-02"
FECHA_FIN="2024-07-01"


In [3]:
os.chdir(RUTA_DATOS)

# Cargar el objeto desde el fichero
with open(FIC_ENTRADA, 'rb') as f:
    df_limpio = pickle.load(f)

## Discretizador de tiempo

Dado que el dataframe original contiene datos diarios, es necesario incrementar la granularidad temporal para obtener lecturas en intervalos más cortos. Para ello se utiliza la función `cortar_y_interpolar_dataset` (que se apoya en `dividir_dia_en_intervalos`), la cual, a partir de una fecha de inicio, fecha de fin y un número de intervalos por día, genera un nuevo dataframe con los registros distribuidos a lo largo del día.  

Aquí lo que vamos a hacer es:

* De todas los días, escoger el rango si queremos. Tanto podemos usar fecha de inicio y fin, como solo inicio o solo fin o no poner nada y así contemplar todos los días.
* Dividir el dataframe en "n" muestras por día.

In [4]:
def dividir_dia_en_intervalos(dataframe, n):
    """
    Divide cada día en `n` intervalos iguales y genera un nuevo DataFrame con un índice datetime.
    
    Parámetros:
        dataframe (pd.DataFrame): DataFrame original con un índice de fechas.
        n (int): Número de intervalos por día.
        
    Retorna:
        pd.DataFrame: DataFrame con un índice datetime que contiene las fechas, horas y las demás columnas.
    """
    if n < 1:
        raise ValueError("El número de muestras 'n' debe ser al menos 1.")
    
    # Verificar que el índice del DataFrame sea datetime
    if not pd.api.types.is_datetime64_any_dtype(dataframe.index):
        dataframe.index = pd.to_datetime(dataframe.index)

    resultados = []

    for fecha, fila in dataframe.iterrows():
        # Generar un rango de tiempo para dividir el día en `n` intervalos
        inicio_dia = pd.Timestamp(fecha)
        fin_dia = inicio_dia + pd.Timedelta(days=1)
        intervalos = pd.date_range(start=inicio_dia, end=fin_dia, periods=n + 1, inclusive="left")

        # Crear un DataFrame temporal para los intervalos del día, replicando las columnas originales
        temp_df = pd.DataFrame({'datetime': intervalos})
        for columna in dataframe.columns:
            temp_df[columna] = fila[columna]
        resultados.append(temp_df)
    
    # Combinar los resultados en un solo DataFrame
    resultado_final = pd.concat(resultados, ignore_index=True)
    
    # Establecer el índice como el datetime generado
    resultado_final.set_index('datetime', inplace=True)
    
    return resultado_final


def cortar_y_interpolar_dataset(dataset, fecha_inicio=None, fecha_fin=None, n=2):
    """
    Corta e interpola un dataset por fecha y un número de muestras diario.
    
    Parámetros:
        dataset (pd.DataFrame): El dataframe con los datos.
        fecha_inicio (str): Fecha inicial en formato "YYYY-MM-DD". Si es None, se toma desde el inicio.
        fecha_fin (str): Fecha final en formato "YYYY-MM-DD". Si es None, se toma hasta el final.
        n (int): Número de muestras diarias deseadas (debe ser >= 2).
    
    Retorna:
        pd.DataFrame: El dataset cortado e interpolado.
    """
    # Validar que n es válido
    if n < 2:
        raise ValueError("El número de muestras 'n' debe ser mayor o igual a 2.")
    
    # Convertir la columna 'fecha' a tipo datetime si no lo está
    if not pd.api.types.is_datetime64_any_dtype(dataset['fecha']):
        dataset['fecha'] = pd.to_datetime(dataset['fecha'])
    
    # Filtrar el dataset según las fechas dadas
    if fecha_inicio:
        fecha_inicio = pd.to_datetime(fecha_inicio)
        if fecha_inicio < dataset['fecha'].min():
            raise ValueError("La fecha de inicio está fuera del rango del dataset.")
        dataset = dataset[dataset['fecha'] >= fecha_inicio]
    
    if fecha_fin:
        fecha_fin = pd.to_datetime(fecha_fin)
        if fecha_fin > dataset['fecha'].max():
            raise ValueError("La fecha de fin está fuera del rango del dataset.")
        dataset = dataset[dataset['fecha'] <= fecha_fin]


    # Agrupar el dataset por día y aplicar la interpolación
    dataset.set_index('fecha', inplace=True)
    #print(dataset)
    #dataset_resultado = dataset.groupby(pd.Grouper(freq='D')).apply(generar_muestras).reset_index(drop=False)

    return dividir_dia_en_intervalos(dataset, n)

In [5]:
df_discretizado = cortar_y_interpolar_dataset(df_limpio, fecha_inicio=FECHA_INICIO, fecha_fin=FECHA_FIN, n=INTERVALOS_TIEMPO_POR_DIA)

#print(df_discretizado.head(50))

## Generador de la velocidad de la máquina

### Escenario general

1. **Funcionamiento continuo**: La empresa opera las 24 horas del día, divididas en 3 turnos de 8 horas cada uno. No se cierra por la noche.
2. **Posibles paradas**: Aunque la producción es continua, cada día existe:
   - Un 5% de probabilidad de **avería** que provoca la parada de la máquina.  
   - Un 30% de probabilidad de **cambio de diseño**, lo que también requiere parar la máquina para hacer ajustes.  

   > **Nota**: Si se produce una parada, habrá que determinar su duración y la posterior puesta a punto antes de volver a producir.

---

### Estados de la máquina

La máquina puede encontrarse en uno de los siguientes cuatro estados de velocidad, con las siguientes restricciones y rangos:

1. **Parada**  
   - Velocidad = 0 m/min (máquina detenida).  
   - Suele ocurrir por mantenimiento, avería, cambios de material o cambios de diseño.  
   - **Transición**: de “Parada” solo se puede pasar a “Puesta a punto” (no directamente a “Normal” ni “Alta”).

2. **Puesta a punto**  
   - Velocidad en el rango **[10, 50] m/min**.  
   - Se utiliza para calibrar la máquina, ajustar registros de color, probar el material, etc.  
   - **Transición**: desde “Puesta a punto” podemos pasar a “Parada”, “Normal” o “Alta”.  
   - **Duración típica**: entre 10 y 60 minutos.  
   - **Variaciones internas**: dentro de una misma puesta a punto, el operario puede cambiar la velocidad (dentro de [10,50]) hasta 3 veces, con una probabilidad de 30% de realizar *cada* cambio.  
   - **Distribución de tiempo**: a lo largo del día, la suma total de puesta a punto no debería superar el 10% del tiempo (aunque puede ser 0% si no se requiere).

3. **Velocidad normal (producción)**  
   - Velocidad fija elegida al **inicio** del estado dentro de [300, 350] m/min.  
   - Una vez seleccionada, se mantiene prácticamente constante (puede tener una variabilidad intrínseca máxima de 1% sobre el valor fijado).  
   - **Transición**: desde “Normal” se puede pasar a “Parada” o “Alta” (pero **no** a “Puesta a punto” directamente).  
   - Se usa para trabajos de impresión que requieren calidad normal.

4. **Velocidad máxima (alta producción)**  
   - Velocidad fija al inicio dentro de un rango cercano a 500 m/min (por ejemplo, [490, 510]).  
   - Una vez fijada, se mantiene constante igual que en “Normal” (sin variación salvo eventos fortuitos).  
   - **Transición**: desde “Alta” se puede pasar a “Parada” o a “Normal” (pero **no** a “Puesta a punto” directamente).  
   - Se utiliza para maximizar la producción en materiales que requieren menos precisión.

---

### Probabilidades y distribución de estados en un día

1. **Parada**  
   - Ocurre si se cumple la avería (5% al día) o si hay un cambio de diseño (30% al día), **o** por paradas planificadas (mantenimiento, cambios de material, etc.).  
   - Si se decide modelar paradas planificadas, se podría forzar un 5-10% del tiempo diario de paro.  
   - Al reanudar, siempre se pasa primero por “Puesta a punto”.

2. **Puesta a punto** (máximo 10% del tiempo diario)  
   - Cuando termina la “Parada” o cuando se hace un cambio importante de material/diseño, se entra en este estado.  
   - La duración de cada puesta a punto está entre 10 y 60 minutos (puede haber varias puestas a punto en un día, si hay varias paradas o cambios).

3. **Producción (Normal o Alta)**  
   - Una vez finalizada la puesta a punto, puede iniciarse la producción.  
   - **Elección**: al comenzar la producción, la probabilidad de optar por “Normal” es 70%, y la probabilidad de optar por “Alta” es 30%.  
   - Así, cada vez que se inicia (o reinicia) la producción, se elige “Normal” con un 70% de probabilidad y “Alta” con un 30%.  
   - Desde “Normal” o “Alta” se puede pasar a “Parada” por avería, fin de trabajo, etc. (en cuyo caso, volveríamos a “Puesta a punto” después de la Parada, si se retoma la producción).

---

### Ruido fortuito (aleatorio)

- Se define un **1% de probabilidad** de ruido **por intervalo** (o por la unidad de tiempo que se decida) **mientras la máquina esté en movimiento** (es decir, en “Puesta a punto”, “Normal” o “Alta”, pero no en “Parada”).  
- Este ruido produce una variación puntual de la velocidad (por ejemplo, ±5%) y dura uno o dos intervalos, tras los cuales la velocidad vuelve a su valor normal.  
- Importante aclarar:  
  - Si vas a simular con “intervalos de 1 minuto” o “intervalos de 15 minutos”, debes aplicar esa probabilidad del 1% en **cada** intervalo en el que la máquina está activa.  
  - Si prefieres interpretarlo como “1% al día de que ocurra un único evento” (menos frecuente), sería un golpe cada 100 días en promedio, lo que quizá sea demasiado bajo.

---

### Aceleración y desaceleración

- **Aceleración (5 a 10 segundos)**: cuando se pasa de un estado de menor velocidad a otro de mayor velocidad (por ejemplo, de “Puesta a punto” a “Normal”), la máquina no salta instantáneamente a la velocidad final.  
  - En la simulación, puede modelarse como un pequeño tramo (en segundos) donde la velocidad sube gradualmente desde la anterior hasta la nueva.  
- **Desaceleración (3 a 6 segundos)**: lo mismo, pero a la inversa, cuando se reduce velocidad (por ejemplo, de “Alta” a “Parada”).

> En caso de que los datos se registren cada 15 minutos, estas aceleraciones de pocos segundos apenas se verán reflejadas.

---

### Variaciones dentro de “Puesta a punto” y “Normal/Alta”

- **Puesta a punto**:
  - Velocidad base fijada al entrar (en [10,50] m/min).  
  - Puede tener hasta 3 *cambios* de velocidad internos, cada uno con 30% de probabilidad.  
  - Cada cambio varía la velocidad en ±20% del valor vigente, siempre manteniéndose en el rango [10,50].

- **Normal y Alta**:
  - Al entrar, se define un valor (entre [300,350] o [490,510], respectivamente).  
  - Se mantiene **sin variación** salvo por el ruido fortuito descrito. 

In [6]:
def generar_estado_y_velocidad_por_dia(df_dia,
                                       prob_averia=0.05,    # 5% prob. al día de avería
                                       prob_cambio=0.30,    # 30% prob. al día de cambio de diseño
                                       max_puesta_pct=0.10, # máx. 10% del tiempo en puesta a punto
                                       dt_min=15,           # intervalo (minutos) entre filas
                                       ):
    """
    Genera dos columnas:
      - 'estado'    (['parada', 'puesta', 'normal', 'alta'])
      - 'velocidad' (float)
    para el DataFrame de UN día (df_dia).

    Parámetros:
      - df_dia: DataFrame con índice datetime correspondiente a un solo día
      - prob_averia: Probabilidad de avería en ese día (0.05 => 5%)
      - prob_cambio: Probabilidad de cambio de diseño en ese día (0.30 => 30%)
      - max_puesta_pct: Máximo % de tiempo (del total del día) en "puesta a punto".
      - dt_min: Cantidad de minutos entre cada fila (en tu caso, 15).
    """
    
    n_intervalos = len(df_dia)  # número de filas en este día
    total_minutos = n_intervalos * dt_min
    
    # ---------------------------------------------------------
    # 1. Determinar si habrá parada por avería o cambio diseño
    # ---------------------------------------------------------
    hay_averia = (np.random.rand() < prob_averia)
    hay_cambio = (np.random.rand() < prob_cambio)
    
    # Si hay avería o cambio, suponemos UNA parada diaria, 15-60 min:
    if hay_averia or hay_cambio:
        duracion_parada = np.random.randint(15, 61)  # entero aleatorio entre 15 y 60
    else:
        duracion_parada = 0

    # Convertir a número de intervalos (usando round)  # <-- CAMBIO CON ROUND()
    intervalos_parada = duracion_parada / dt_min
    intervalos_parada = int(round(intervalos_parada))
    intervalos_parada = max(0, min(intervalos_parada, n_intervalos))  # no exceder [0, n_intervalos]
    
    # ---------------------------------------------------------
    # 2. Determinar la duración total de puesta a punto
    # ---------------------------------------------------------
    max_minutos_puesta = total_minutos * max_puesta_pct
    if max_minutos_puesta < 10:
        # No habrá puesta a punto si ni siquiera llega a 10 min
        minutos_puesta = 0
    else:
        # Si hubo parada, forzamos puesta a punto a continuación (10-60 min)
        minutos_puesta = 0
        if hay_averia or hay_cambio:
            # límite superior: min(60, max_minutos_puesta)
            limite_superior = int(min(60, max_minutos_puesta))
            if limite_superior < 10:
                # si se diera el caso raro de que max_minutos_puesta < 10,
                # quedaría 0, pero ya hicimos un check antes.
                limite_superior = 0
            if limite_superior > 0:
                minutos_puesta = np.random.randint(10, limite_superior + 1)

    # Convertir a intervalos (usando round)  # <-- CAMBIO CON ROUND()
    intervalos_puesta = minutos_puesta / dt_min
    intervalos_puesta = int(round(intervalos_puesta))
    intervalos_puesta = max(0, min(intervalos_puesta, n_intervalos - intervalos_parada))
    
    # ---------------------------------------------------------
    # 3. Producción (resto del tiempo)
    # ---------------------------------------------------------
    intervalos_produccion = n_intervalos - (intervalos_parada + intervalos_puesta)
    
    # ---------------------------------------------------------
    # 4. Construir la secuencia de estados (por fila)
    # ---------------------------------------------------------
    estados = []
    
    # Bloque 1: PARADA
    for _ in range(intervalos_parada):
        estados.append('parada')
    
    # Bloque 2: PUESTA A PUNTO
    for _ in range(intervalos_puesta):
        estados.append('puesta')
    
    # Bloque 3: PRODUCCIÓN (si queda tiempo)
    # Elegimos aleatoriamente: 70% normal, 30% alta
    if intervalos_produccion > 0:
        if np.random.rand() < 0.70:
            estado_produccion = 'normal'
        else:
            estado_produccion = 'alta'
        
        for _ in range(intervalos_produccion):
            estados.append(estado_produccion)
    
    # Ajustar si nos pasamos
    estados = estados[:n_intervalos]
    
    # Si nos quedamos cortos, rellenar con 'normal'
    while len(estados) < n_intervalos:
        estados.append('normal')
    
    # ---------------------------------------------------------
    # 5. Generar la velocidad base para cada estado
    # ---------------------------------------------------------
    velocidades = np.zeros(n_intervalos, dtype=float)
    
    for i, est in enumerate(estados):
        if est == 'parada':
            velocidades[i] = 0.0
        elif est == 'puesta':
            # Elige velocidad inicial [10..50]
            velocidades[i] = np.random.uniform(10, 50)
        elif est == 'normal':
            # Velocidad [300..350]
            velocidades[i] = np.random.uniform(300, 350)
        elif est == 'alta':
            # Velocidad [490..510]
            velocidades[i] = np.random.uniform(490, 510)
    
    # ---------------------------------------------------------
    # 6. Variaciones internas en puesta a punto
    # ---------------------------------------------------------
    for i, est in enumerate(estados):
        if est == 'puesta':
            # Hasta 3 cambios, 30% prob, ±20%
            for _ in range(3):
                if np.random.rand() < 0.30:
                    factor = 1.0 + np.random.uniform(-0.2, 0.2)
                    velocidades[i] *= factor
                    velocidades[i] = np.clip(velocidades[i], 10, 50)
    
    # ---------------------------------------------------------
    # 7. Aceleración / Desaceleración (muy simplificado)
    # ---------------------------------------------------------
    vel_final = velocidades.copy()
    
    for i in range(1, n_intervalos):
        if estados[i] != estados[i-1]:
            v_prev = vel_final[i-1]
            v_curr = vel_final[i]
            # Ajustamos al 50% entre ambos
            vel_final[i] = (v_prev + v_curr) / 2.0
    
    # ---------------------------------------------------------
    # 8. Ruido fortuito (1% por intervalo activo)
    # ---------------------------------------------------------
    for i, est in enumerate(estados):
        if est != 'parada':
            if np.random.rand() < 0.01:
                # Golpe ±5%
                delta = np.random.uniform(-0.05, 0.05)
                vel_final[i] *= (1 + delta)
                if i+1 < n_intervalos and estados[i+1] != 'parada':
                    vel_final[i+1] *= (1 + delta/2)
    
    # Ajustar rangos finales
    for i, est in enumerate(estados):
        if est == 'parada':
            vel_final[i] = 0.0
        elif est == 'puesta':
            vel_final[i] = np.clip(vel_final[i], 10, 50)
        elif est == 'normal':
            vel_final[i] = np.clip(vel_final[i], 300, 350)
        elif est == 'alta':
            vel_final[i] = np.clip(vel_final[i], 490, 510)
    
    # Construir DataFrame resultante con columnas nuevas
    df_result = df_dia.copy()
    df_result['estado'] = estados
    df_result['velocidad'] = vel_final
    
    return df_result


def agregar_velocidad_y_estado(df):
    """
    Función principal que agrupa el DataFrame por día y
    aplica generar_estado_y_velocidad_por_dia a cada grupo.
    
    Devuelve el DataFrame original con 2 columnas nuevas:
      - 'estado'
      - 'velocidad'
    """
    df_out = []
    
    # Verificar la frecuencia (en minutos) entre dos filas consecutivas.
    # Asumimos que es constante.
    if len(df) < 2:
        dt_min = 15
    else:
        dt_min = (df.index[1] - df.index[0]).total_seconds() / 60.0
    
    # Agrupamos por día
    grouped = df.groupby(df.index.date)
    
    for date_val, df_dia in grouped:
        df_dia = df_dia.sort_index()
        
        df_dia_res = generar_estado_y_velocidad_por_dia(
            df_dia,
            prob_averia=0.05,
            prob_cambio=0.30,
            max_puesta_pct=0.10,
            dt_min=dt_min,   # usaremos este dt_min calculado
        )
        df_out.append(df_dia_res)
    
    df_final = pd.concat(df_out).sort_index()
    
    return df_final


In [7]:
df_velocidad = agregar_velocidad_y_estado(df_discretizado)
print(df_velocidad['estado'].unique())
print(df_velocidad.head())

['parada' 'puesta' 'normal' 'alta']
                         tmin  tmed  tmax  horatmin  horatmax  hrMin  hrMedia  \
datetime                                                                        
2023-01-02 00:00:00.000   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:02:52.800   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:05:45.600   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:08:38.400   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:11:31.200   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   

                         hrMax horaHrMin horaHrMax  estado  velocidad  
datetime                                                               
2023-01-02 00:00:00.000   93.0  00:00:00  23:59:00  parada        0.0  
2023-01-02 00:02:52.800   93.0  00:00:00  23:59:00  parada        0.0  
2023-01-02 00:05:45.600   93.0  00:00:00  23:59:00  parada        0.0  
2023-01-02 00:08:38.400   93.0  00:0

### Representación gráfica

Como hay muchos datos, usaré **plotly** ya que es interactivo y visualizaré el gráfico en una pestaña del navegador.

In [8]:
#import plotly.express as px
#import plotly.io as pio

# Configurar el renderizador para JupyterLab
#pio.renderers.default = "browser"

# Graficar
#fig = px.line(df_velocidad, x=df_velocidad.index, y=df_velocidad["velocidad"], title="Velocidad en función del tiempo (Interactivo)")
#fig.update_xaxes(title="Tiempo")
#fig.update_yaxes(title="Velocidad (metros/minuto)")
#fig.show()


## Añadir COLOR

Para simular el color impreso a lo largo del tiempo, se empleará el **espacio de color LAB** con rangos específicos para \(L\) (0 a 100), \(A\) (≈ -128 a +127) y \(B\) (≈ -128 a +127). Cada **patrón de impresión** podrá tener sus propios **umbrales** y ajustes de tolerancia en \(L, A, B\). Estos umbrales se ven porque la máquina pasa a estado de *parada* aunque el paso a este estado no solo es por eso sino que puede deberse a cambio de patrones, averías...

Además, se contemplarán **factores** que influyen en la evolución del color:

1. **Aceleraciones y desaceleraciones**:  
   - Los cambios bruscos de velocidad pueden causar **desajustes temporales** de color, simulados como picos o desvíos momentáneos.

2. **Humedad y temperatura** (NO contemplado):  
   - No los voy a contemplar debido a que es muy difícil simularlos y en un inicio las máquinas no cuentan con ningún dispositivo de este tipo.

3. **Deriva por tiempo prolongado**:  
   - Con el paso de los minutos u horas en un mismo estado (p. ej. velocidad normal o alta), el color se **desvía gradualmente** debido a saturación, desgastes, etc.  
   - Al entrar en “puesta a punto”, se **reajusta** el color a su valor original.

4. **Ruido aleatorio**:  
   - Para reflejar la **imprecisión del sensor** o fluctuaciones reales, se añadirán variaciones pequeñas y aleatorias a \(L, A, B\).

El **pipeline** de simulación parte de un **color base** (que depende del estado y patrón de diseño) y luego aplica correcciones en cada intervalo:
- **Aceleración/Desaceleración**  
- **Efecto Humedad/Temperatura** (No contemplado)
- **Deriva por tiempo**
- **Ruido aleatorio**
- **Clip** de valores para no salir de los límites de LAB

### Función `_obtener_lab_base`

Esta función se encarga de generar (o inicializar) los valores de color **L**, **A** y **B** para cada fila de un DataFrame a partir de la columna `'estado'`. La lógica tiene en cuenta distintas condiciones:

1. **Máquina parada**  
   - Cuando la máquina está en estado `'parada'`, no se asocia ningún color (los valores de `L`, `A` y `B` se establecen a `NaN`).
   
2. **Transición desde `'parada'`**  
   - Al pasar de `'parada'` a cualquier otro estado ( `'puesta'`, `'normal'` o `'alta'` ), existe un 50% de probabilidad de **mantener el color anterior** (si se conocía uno) y un 50% de probabilidad de **generar un nuevo color** (vistoso). Si cambiamos el color sería la simulación de que se ha cambiado el patrón o diseño a imprimir.
   - Si la máquina no tenía un color anterior (por ejemplo, si es la primera vez que deja de estar parada), se genera uno nuevo de forma determinista.

3. **Mantener color en otros casos**  
   - Si se permanece o se transiciona entre los estados `'puesta'`, `'normal'` o `'alta'` (es decir, no se viene de `'parada'`), se **mantiene** el color actual a menos que no existiera (en cuyo caso, se genera uno nuevo).

4. **Color vistoso**  
   - Un color vistoso es aquel cuyos valores de **L** se encuentran en un rango de luminosidad [50,80], mientras que **A** y **B** se encuentran en el rango [-80,80].  
   - Esto proporciona una mayor saturación y brillo, de modo que sea un color “llamativo”.

5. **Almacenar y reutilizar color anterior**  
   - Se mantiene en memoria (`color_anterior_no_parada`) el último color activo (no-`NaN`) antes de que la máquina se haya parado. De esta forma, al reanudar la producción o comenzar la puesta a punto, puede decidirse entre reutilizarlo o actualizarlo.

#### Flujo Interno

1. Se recorren las filas del DataFrame según el índice.  
2. Para cada fila, se detecta el estado (`'parada'`, `'puesta'`, `'normal'`, `'alta'`) y se aplican las reglas anteriores para asignar los valores de **L**, **A** y **B**.  
3. Se almacenan estos valores en listas y luego se añaden al DataFrame como columnas `"L"`, `"A"`, y `"B"`.

#### Resultado

- El DataFrame de salida tendrá, en cada fila, valores de **L, A, B**:
  - `NaN` en caso de `'parada'`.
  - Un color vistoso (fijo o recién generado) en los demás estados.


In [9]:
def _obtener_lab_base(df):
    """
    Genera (inicializa) las columnas L, A, B en función de los estados,
    cumpliendo:
      - Cuando la máquina está 'parada', no hay color (NaN).
      - Cuando pasa a 'puesta', si venimos de 'parada', 50% de
        mantener el color anterior (si existía) y 50% de obtener uno nuevo.
      - Si venimos de 'normal', 'alta' o 'puesta' a 'puesta', 
        se mantiene el mismo color que ya teníamos.
    """

    # Función auxiliar para generar un color vistoso
    def _generar_color_vistoso():
        # Ejemplo: L ∈ [50..80], A, B ∈ [-80..80]
        L = np.random.uniform(50, 80)
        A = np.random.uniform(-80, 80)
        B = np.random.uniform(-80, 80)
        return (L, A, B)

    df_out = df.copy()

    # Listas donde iremos guardando los valores de L, A, B fila a fila
    L_list = []
    A_list = []
    B_list = []

    # Variables para llevar el "estado anterior" y "color anterior"
    estado_anterior = None
    # El color anterior es aquel que se usaba antes de parar (o sea, el último color
    # en un estado distinto de 'parada').
    color_anterior_no_parada = None

    # Variable para almacenar el color "actual" en la fila i
    current_color = None

    for i, row in df_out.iterrows():
        estado_actual = row['estado']

        if estado_actual == 'parada':
            # Si la máquina está parada, NO hay color
            current_color = (np.nan, np.nan, np.nan)

        elif estado_actual == 'puesta':
            # Si venimos de "parada", 50% de mantener el color_anterior_no_parada
            # y 50% de cambiarlo por uno nuevo. (Si color_anterior_no_parada es None,
            # entonces generamos uno nuevo siempre).
            if estado_anterior == 'parada':
                # color_anterior_no_parada podría ser None si nunca se tuvo un color.
                if color_anterior_no_parada is not None:
                    if np.random.rand() < 0.5:
                        # Mantenemos el color anterior
                        current_color = color_anterior_no_parada
                    else:
                        # Generamos nuevo
                        current_color = _generar_color_vistoso()
                else:
                    # No existe color anterior, así que generamos uno nuevo
                    current_color = _generar_color_vistoso()

            else:
                # Si venimos de 'normal', 'alta' o 'puesta' a 'puesta',
                # mantenemos el color actual (si lo tuviéramos)
                if current_color is None or pd.isna(current_color[0]):
                    # Si no tenemos color, generamos uno
                    current_color = _generar_color_vistoso()
                # Si current_color existe, lo mantenemos tal cual

        else:
            # estados = 'normal' o 'alta'
            # Si venimos de 'parada', no hay color actual, pero hay 50% 
            # de mantener el color_anterior_no_parada.
            if estado_anterior == 'parada':
                if color_anterior_no_parada is not None:
                    if np.random.rand() < 0.5:
                        current_color = color_anterior_no_parada
                    else:
                        current_color = _generar_color_vistoso()
                else:
                    # No existe color anterior, generamos uno
                    current_color = _generar_color_vistoso()
            else:
                # Si ya estábamos en 'puesta', 'normal' o 'alta', mantenemos color
                if current_color is None or pd.isna(current_color[0]):
                    current_color = _generar_color_vistoso()

        # Añadimos a las listas
        L_list.append(current_color[0])
        A_list.append(current_color[1])
        B_list.append(current_color[2])

        # Si el estado actual NO es 'parada', entonces actualizamos color_anterior_no_parada
        if estado_actual != 'parada':
            color_anterior_no_parada = current_color

        # Actualizamos estado_anterior
        estado_anterior = estado_actual

    # Añadimos las columnas al DF
    df_out['L'] = df_out['L_base'] = L_list
    df_out['A'] = df_out['A_base'] = A_list
    df_out['B'] = df_out['B_base'] = B_list

    return df_out


### Función `_aplicar_aceleracion_desaceleracion`

Esta función **ajusta** los valores de **L, A y B** de un DataFrame en función de las **aceleraciones** o **desaceleraciones bruscas** de la máquina. El objetivo es **simular** (de manera aproximada) la influencia física que tienen los cambios rápidos de velocidad sobre el color:

- **Aceleraciones** positivas (aumento de velocidad por encima de cierto umbral) pueden provocar ligeras variaciones que tienden a “desaturar” el color (A y B se acercan a 0) y/o aumentar un poco la luminosidad (L).  
- **Desaceleraciones** bruscas (descenso de velocidad por encima de un umbral) pueden provocar acumulaciones de tinta y oscurecer el color (bajar L) y/o incrementar la saturación (subir A y B).

#### Parámetros de la función

- **df** (`pd.DataFrame`):  
  DataFrame que **debe** contener las columnas:
  - `'estado'`: para ignorar el ajuste cuando la máquina está parada.
  - `'L'`, `'A'`, `'B'`: valores de color ya asignados antes de aplicar la física de la aceleración/desaceleración.
  - `'velocidad'` (o la que se indique en `velocidad_col`): valores de velocidad en cada intervalo.

- **velocidad_col** (`str`, por defecto `'velocidad'`):  
  Nombre de la columna en `df` que representa la velocidad.  

- **threshold** (`float`, por defecto `50`):  
  Umbral de **cambio de velocidad** a partir del cual se considera que hay una aceleración/desaceleración **brusca**. Ejemplo: `50` (m/min).  

- **factor_acel** (`float`, por defecto `0.3`):  
  Factor de ajuste aplicado cuando `delta_v` (cambio de velocidad) es **positivo** y supera el umbral. Controla la **magnitud** del efecto sobre L, A, B.  

- **factor_decel** (`float`, por defecto `0.3`):  
  Factor de ajuste aplicado cuando `delta_v` es **negativo** y supera el umbral. Controla la **magnitud** del efecto sobre L, A, B.

#### Comportamiento interno

1. **Recorrido del DataFrame**:  
   Se itera fila a fila, comenzando en la segunda (índice `i=1`), pues se compara la velocidad actual (`vel[i]`) con la anterior (`vel[i-1]`).

2. **Cálculo de `delta_v`**:  
   - `delta_v = vel[i] - vel[i-1]`.  
   - Si `|delta_v| < threshold`, se ignora porque se considera un cambio de velocidad **no** suficientemente grande.

3. **Aceleración (delta_v > 0)**:  
   - Se calcula la “intensidad” como `accel_intensity = factor_acel * (delta_v - threshold)`.  
   - Se incrementa ligeramente `L` (la impresión se aclara) y se mueven A y B hacia 0 (desaturación).  
   - Esto simula que, al acelerar, la tinta no llega a depositarse con la misma densidad.

4. **Desaceleración (delta_v < 0)**:  
   - Se calcula `decel_intensity = factor_decel * (|delta_v| - threshold)`.  
   - Se reduce `L` (la impresión se oscurece) y se incrementa la saturación de A y B (se separan un poco más de 0).  
   - Esto simula que, al frenar, hay tendencia a mayor acumulación de tinta.

5. **Máquina parada**:  
   - Si en la fila `i` el estado es `'parada'`, no se aplica ningún ajuste, pues no hay color efectivo ni efecto de aceleración.

6. **Actualización de L, A, B**:  
   - Se modifica directamente el valor en la fila `i` para reflejar el resultado.  
   - Al finalizar, las columnas `'L'`, `'A'`, `'B'` quedan con los nuevos valores ajustados.

In [10]:
def _aplicar_aceleracion_desaceleracion(df, 
                                        velocidad_col='velocidad',
                                        threshold=50, 
                                        factor_acel=0.3, 
                                        factor_decel=0.3):
    """
    Ajusta los valores de L, A, B en función de los cambios bruscos de velocidad,
    simulando la influencia de la aceleración/desaceleración sobre el color.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que debe contener al menos las columnas:
        - 'estado': para saber si la máquina está parada.
        - velocidad_col (por defecto 'velocidad'): con los valores de velocidad.
        - 'L', 'A', 'B': columnas con el color base ya asignado.
    velocidad_col : str
        Nombre de la columna en df que representa la velocidad (por defecto 'velocidad').
    threshold : float
        Diferencia mínima en la velocidad (entre dos intervalos consecutivos) que consideramos
        "aceleración/desaceleración brusca". Por ejemplo, 50 m/min de cambio.
    factor_acel : float
        Factor que multiplica la magnitud de la aceleración positiva para cambiar el color.
    factor_decel : float
        Factor que multiplica la magnitud de la aceleración negativa (desaceleración) para cambiar el color.

    Returns
    -------
    df_out : pd.DataFrame
        Mismo DataFrame con los valores de 'L', 'A', 'B' modificados
        según la aceleración/desaceleración en cada intervalo.
    """

    df_out = df.copy()

    # Por comodidad, definimos arrays para manipular L, A, B sin copy-on-write penalidad
    L_arr = df_out['L'].values
    A_arr = df_out['A'].values
    B_arr = df_out['B'].values
    
    vel = df_out[velocidad_col].values

    for i in range(1, len(df_out)):
        # Si la máquina está parada, no aplicamos nada
        # (tampoco hay "aceleración" si no se retoma la marcha).
        if df_out['estado'].iloc[i] == 'parada':
            continue

        # Calculamos delta_v (cambio de velocidad respecto al intervalo anterior)
        delta_v = vel[i] - vel[i-1]

        # Si no supera threshold, consideramos que es un cambio ligero, no hay gran efecto
        if abs(delta_v) < threshold:
            continue

        # Aceleración brusca (delta_v > 0)
        if delta_v > 0:
            # Por ejemplo: una subida brusca de velocidad
            # Podríamos suponer que la tinta se "tensa" menos tiempo y 
            # que produce un leve cambio en saturación. 
            # Aquí, se hace un cambio lineal relativo a delta_v.
            # Con factor_acel controlamos cuánto "pesa" ese cambio de color.

            # Ejemplo: L sube un poco, A y B se acercan a cero (desaturado),
            # con magnitud relativa a delta_v.
            accel_intensity = factor_acel * (delta_v - threshold)
            # Ajustamos L: subimos luminosidad
            L_arr[i] += +0.1 * accel_intensity
            # Ajustamos A, B: nos movemos un poco hacia 0
            A_arr[i] -= 0.05 * accel_intensity * np.sign(A_arr[i])
            B_arr[i] -= 0.05 * accel_intensity * np.sign(B_arr[i])

        # Desaceleración brusca (delta_v < 0)
        else:
            decel_intensity = factor_decel * (abs(delta_v) - threshold)
            # Ejemplo: al frenar, la tinta puede acumularse un poco 
            # => bajamos un poco L (más oscuro) y saturamos A, B algo más.
            L_arr[i] -= 0.1 * decel_intensity
            A_arr[i] += 0.05 * decel_intensity * np.sign(A_arr[i])
            B_arr[i] += 0.05 * decel_intensity * np.sign(B_arr[i])
    
    # Asignamos de nuevo los valores ajustados a las columnas
    df_out['L'] = L_arr
    df_out['A'] = A_arr
    df_out['B'] = B_arr

    return df_out


### Función `_aplicar_deriva_tiempo`

Esta función **simula** la **deriva gradual del color** (columnas `L`, `A`, `B`) en aquellos casos donde la máquina lleva un período prolongado imprimiendo en el mismo estado de producción (`normal` o `alta`). El objetivo es reflejar el **cambio progresivo** de color que ocurre por desgaste, saturación o pequeñas variaciones del proceso cuando se imprime sin reajustes durante un tiempo.

#### Comportamiento según el estado

1. **Parada**  
   - La máquina no imprime color (generalmente `NaN` en `L`, `A`, `B`).  
   - No se acumula ninguna deriva durante la parada.

2. **Puesta a punto**  
   - Se asume que en este estado el operario **reajusta** o **calibra** el color al valor “base” que ya se estableció al entrar en puesta (p. ej. desde `_obtener_lab_base`).  
   - Por tanto, **no** se aplica deriva alguna ni se acumula con el tiempo.  
   - Sirve como “reseteo” de cualquier desviación anterior.

3. **Normal** o **Alta**  
   - Aquí se **acumula** la deriva con cada intervalo (fila) que el estado se mantenga de forma consecutiva.  
   - El **contador** de filas consecutivas en el mismo estado se incrementa en cada iteración. Luego se ajusta `(L, A, B)` con pequeños incrementos o decrementos, configurados a través de parámetros de la función (`drift_factor_L`, `drift_factor_A`, `drift_factor_B`).

4. **Cambio de Normal a Alta (o viceversa)**  
   - Se mantiene el color (con el posible ajuste de aceleración/desaceleración aplicado anteriormente) como punto de partida del nuevo estado.  
   - Se **reinicia** el contador de “filas en el mismo estado” para comenzar a acumular deriva nuevamente en el estado entrante.

#### Parámetros de la función

- **df** (`pd.DataFrame`):  
  DataFrame con las columnas:
  - `'estado'`: indica si la máquina está en `'parada'`, `'puesta'`, `'normal'` o `'alta'`.  
  - `'L'`, `'A'`, `'B'`: valores de color antes de aplicar la deriva (asignados por pasos previos).  

- **drift_factor_L**, **drift_factor_A**, **drift_factor_B** (`float`), *opcionales*:  
  Parámetros que **controlan** la **velocidad** o **magnitud** de la deriva en cada fila consecutiva en el estado `normal` o `alta`.  
  - Ejemplo: si `drift_factor_L = 0.02`, tras 5 filas continuas en el mismo estado, se sumaría un total de `0.1` a L (0.02 × 5).

#### Forma de la deriva

La función recorre todas las filas (orden cronológico). Para cada fila:
1. **Detecta** el estado (`estado_actual`).  
2. **Mantiene** un contador de “filas consecutivas” (`filas_en_estado`).  
3. **Aplica** (o no) la deriva:
   - Si `estado_actual` es `'normal'` o `'alta'`, **incrementa** `filas_en_estado` y ajusta `(L, A, B)` sumando `drift_factor * filas_en_estado`.  
   - Si `estado_actual` es `'puesta'`, el color se mantiene en el valor base (no se modifica) y se resetea `filas_en_estado`.  
   - Si `estado_actual` es `'parada'`, no hay color, por lo que no hay deriva (y normalmente `L`, `A`, `B` son `NaN`).

#### Resultado

Devuelve un **DataFrame** donde los valores de **L, A, B** se han **modificado** para incorporar la **desviación progresiva** mientras el estado está en `normal` o `alta`. En `puesta`, se respeta el color base (sin cambio), y en `parada`, no se hace nada.

Esta función, integrada en tu **pipeline** de simulación, te permite reflejar el **efecto real** de que, a medida que transcurre el tiempo de impresión en un mismo estado, el color va variando ligeramente hasta que se realice una puesta a punto que lo retorne a los valores deseados.


In [11]:
def _aplicar_deriva_tiempo(df, 
                           drift_factor_L=0.02, 
                           drift_factor_A=0.01, 
                           drift_factor_B=0.01):
    """
    Aplica una deriva gradual al color (L, A, B) cuando la máquina
    está en 'normal' o 'alta', según cuántas filas consecutivas
    lleva en ese estado.
    
    Reglas:
      - Parada: No se hace nada (NaN en L, A, B).
      - Puesta: Se asume que el color se recalibra al valor base
                ya asignado (no se modifica).
      - Normal o Alta:
         * Se incrementa (o decrementa) L, A, B en función de
           cuántas filas consecutivas lleva en el mismo estado.
      - Cambio de normal <-> alta:
         * Mantiene el color alcanzado en la última fila,
           pero el contador de filas en estado se resetea.
    
    Parámetros
    ----------
    df : pd.DataFrame
        Debe contener columnas 'estado', 'L', 'A', 'B', etc.
    drift_factor_L, drift_factor_A, drift_factor_B : float
        Factores de deriva por fila, ajustables según la
        magnitud de desviación deseada.
    
    Retorna
    -------
    df_out : pd.DataFrame
        DataFrame con 'L', 'A', 'B' ajustados para reflejar
        la deriva en el tiempo.
    """

    df_out = df.copy()
    
    # Arrays para manipulación rápida
    L_arr = df_out['L'].values
    A_arr = df_out['A'].values
    B_arr = df_out['B'].values
    
    # Variables para llevar el estado anterior y cuántas filas consecutivas
    # llevamos en 'normal' o 'alta'
    estado_anterior = None
    filas_en_estado = 0  # contador de filas consecutivas en el mismo estado productivo
    
    for i in range(len(df_out)):
        estado_actual = df_out['estado'].iloc[i]
        
        if estado_actual in ['normal', 'alta']:
            # Ver si seguimos en el mismo estado que la fila anterior
            if estado_actual == estado_anterior:
                # Aumentamos contador de filas en este estado
                filas_en_estado += 1
            else:
                # Reseteamos el contador
                filas_en_estado = 0
            
            # Aplicamos deriva sólo si NO estamos en la fila 0 del cambio
            # (es decir, se empieza a notar a partir de la segunda fila en el mismo estado)
            if filas_en_estado > 0:
                # Aumenta la deriva en L, A, B
                # Ejemplo: incrementos lineales
                L_arr[i] += drift_factor_L * filas_en_estado
                A_arr[i] += drift_factor_A * filas_en_estado * np.sign(A_arr[i])
                B_arr[i] += drift_factor_B * filas_en_estado * np.sign(B_arr[i])
            
        elif estado_actual == 'parada':
            # No hay color, no hacemos nada
            # Y no afecta al "estado_anterior" ni filas_en_estado
            pass
        
        elif estado_actual == 'puesta':
            # El color se "resetea" al valor base (el que ya esté en df['L'], etc.)
            # => no hacemos nada para modificarlo, y no continúa la deriva.
            # Rompemos el contador de filas_en_estado:
            filas_en_estado = 0
        
        # Actualizamos estado_anterior
        estado_anterior = estado_actual
    
    # Volvemos a asignar
    df_out['L'] = L_arr
    df_out['A'] = A_arr
    df_out['B'] = B_arr

    return df_out


### Función principal para el cálculo de color `calcular_color_lab`

In [12]:
def calcular_color_lab(df, 
                       consider_humidity=False, 
                       consider_temperature=False):
    """
    Calcula o simula los valores LAB para cada fila del DataFrame 'df'.
    
    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene las columnas (p.ej. estado, velocidad, humedad, temperatura, etc.).
    consider_humidity : bool, opcional
        Indica si se considera el efecto de la humedad en el cálculo de LAB.
    consider_temperature : bool, opcional
        Indica si se considera el efecto de la temperatura en el cálculo de LAB.

    Returns
    -------
    df_out : pd.DataFrame
        El DataFrame con las columnas 'L', 'A', 'B' calculadas (o generadas).
    """

    # Copiamos el DataFrame original para no modificarlo en sitio
    df_out = df.copy()

    # 1. Generar columna base de L, A, B
    df_out = _obtener_lab_base(df_out)

    # 2. Aplicar efectos de aceleración / desaceleración
    df_out = _aplicar_aceleracion_desaceleracion(df_out)

    # 3. (Opcional) Efecto de la humedad
    #if consider_humidity:
    #    df_out = _aplicar_efecto_humedad(df_out)

    # 4. (Opcional) Efecto de la temperatura
    #if consider_temperature:
    #    df_out = _aplicar_efecto_temperatura(df_out)

    # 5. Aplicar deriva con el paso del tiempo
    df_out = _aplicar_deriva_tiempo(df_out)

    # 6. Clip o limitar los valores finales de L, A, B (si fuera necesario)
    #df_out = _clamp_valores_lab(df_out)

    return df_out


In [13]:
df_color = calcular_color_lab(df_velocidad)
print(df_color)

                         tmin  tmed  tmax  horatmin  horatmax  hrMin  hrMedia  \
datetime                                                                        
2023-01-02 00:00:00.000   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:02:52.800   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:05:45.600   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:08:38.400   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:11:31.200   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
...                       ...   ...   ...       ...       ...    ...      ...   
2024-07-01 23:45:36.000   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 23:48:28.800   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 23:51:21.600   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 23:54:14.400   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 23:57:07.200   9.

## Gráfica del color

In [14]:
pio.renderers.default = "browser"

def plot_color_and_velocity(df):
    """
    Crea y muestra un gráfico con dos subplots:
     1. Velocidad vs. tiempo
     2. L, A y B vs. tiempo
    usando Plotly.
    
    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame con índice datetime y columnas:
        - 'velocidad': valores de velocidad en cada instante
        - 'L', 'A', 'B': valores de color LAB en cada instante.
        
    Retorno
    -------
    fig : go.Figure
        Objeto Figure de Plotly para que puedas mostrarlo o modificarlo.
    """

    # Creamos la figura con 2 subplots (comparten el eje X de tiempo).
    fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=("Velocidad", "Color (L, A, B)")
    )

    # Subplot 1 (fila 1): Velocidad
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['velocidad'], 
            mode='lines',
            name='Velocidad',
            line=dict(color='purple')
        ),
        row=1, col=1
    )

    # Subplot 2 (fila 2): L, A, B
    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['L'], 
            mode='lines',
            name='L',
            line=dict(color='blue')
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['A'], 
            mode='lines',
            name='A',
            line=dict(color='red')
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df.index, 
            y=df['B'], 
            mode='lines',
            name='B',
            line=dict(color='green')
        ),
        row=2, col=1
    )

    # Ajustes finales del layout
    fig.update_layout(
        title="Evolución de Velocidad y Color (LAB) en el tiempo",
        height=600,
        xaxis2_title="Tiempo",  # Mover la etiqueta del eje X al último subplot
    )

    # Mostramos el gráfico (en Jupyter, basta fig.show())
    fig.show()
    
    # Devolvemos la figura si se quiere seguir manipulando
    return fig


In [15]:
fig = plot_color_and_velocity(df_color)

## Enriquecimiento de los datos

Como ya he mencionado, actualmente la máquina solo nos provee de los siguientes atributos:

* Velocidad
* LAB deseado
* LAB real
* Tiempo

En este punto y para ayudar al modelo o modelos a converger, me he reunido con el cliente e investigado los datos y creemos que los siguientes valores pueden aportar valor. Es evidente que una vez tenga los datos enriquecidos, podré luego descartarlos sino me interesan. De momento voy a incorporar los siguientes:

* Δ𝐸
* Patrón
* Momento del día
* Duración de la parada anterior
* Tiempo en cada estado

### Incorporar el indicador `Δ𝐸` al dataframe

Según el cliente añadir este indicador puede ser útil ya que el indicador `Δ𝐸` es fundamental cuando se necesita que las métricas reflejen diferencias perceptibles para el ojo humano. EL uso de `ΔE` junto con métricas como RMSE te permitirá una evaluación más completa y práctica del desempeño de los modelos.

Para calcular este valor usaremos la fórmula CIE76 que es apropiada para nuestro caso según el cliente.

El CIE76 es una fórmula estandarizada por la Comisión Internacional de Iluminación (CIE) en 1976 para calcular la diferencia entre dos colores en el espacio de color CIE LAB. Es la versión más simple de la métrica `ΔE`, que mide cuán diferentes son dos colores desde una perspectiva perceptual.

La percepción de cuánto ha variado un color depende del **(ΔE)** y de la **tolerancia** que se maneje en cada industria o aplicación. Sin embargo, a nivel general (en especial para **CIE76**), suelen emplearse los siguientes **umbrales orientativos**:

- (ΔE < 1):  
  Prácticamente imperceptible para el ojo humano en condiciones normales.  
- (1 <= ΔE < 2):  
  Ligeramente perceptible para observadores con alta sensibilidad.  
- (2 <= ΔE < 3):  
  Perceptible, pero puede considerarse aceptable para muchas aplicaciones.  
- (3 <= ΔE < 5):  
  Diferencia claramente apreciable.  
- (ΔE >= 5):  
  Variación grande; se considera un cambio de color bastante notorio.

En **impresión** y **control de calidad**, a menudo se usa un **(Delta E) de 2 o 3** como límite máximo de tolerancia. Por encima de ese valor, la mayoría de los clientes y técnicos ya consideran que el color se ha desviado demasiado.

In [16]:
# Calcular (Delta E) (usando CIE76)
def delta_e_cie76(row):
    delta_L = row['L'] - row['L_base']
    delta_A = row['A'] - row['A_base']
    delta_B = row['B'] - row['B_base']
    return np.sqrt(delta_L**2 + delta_A**2 + delta_B**2)

# Aplicar la función al DataFrame
df_color['Delta_E'] = df_color.apply(delta_e_cie76, axis=1)

### Incorporar el patrón

Como la máquina es capaz de imprimir diferentes colores en diferentes momentos, los modelos verán cambios abruptos, y en según que casos puede ayudar incorporar una columna con un identificador para cada patrón diferente. Si la máquina repite patrón en otro momento, el identificador se repetirá.

In [17]:
def asignar_patron_id(df, cols_base=('L_base','A_base','B_base')):
    """
    Crea una columna 'patron_id' en el df a partir de la combinación 
    única de (L_base, A_base, B_base).
    
    Param:
    ------
    df : pd.DataFrame
        Debe contener las columnas L_base, A_base, B_base.
    cols_base : tuple
        Nombres de columnas que definen el patrón.

    Returns:
    --------
    df_out : pd.DataFrame
        Mismo DataFrame con una columna nueva 'patron_id' (int).
    """
    df_out = df.copy()

    # 1. Convertir a string o a tuplas para agrupar
    #    Por ejemplo, la tupla (L_base, A_base, B_base)
    #    Se crea una serie con dichas tuplas
    base_tuples = df_out[list(cols_base)].apply(tuple, axis=1)

    # 2. Obtener categoría única
    #    factorize crea IDs de 0..n-1 para cada valor único
    patron_ids, uniques = pd.factorize(base_tuples)

    df_out['patron_id'] = patron_ids  # entero que identifica cada patrón

    return df_out


In [18]:
df_color = asignar_patron_id(df_color)

### Incorporar momento del día

Para evitar ambigüedades y después de investigar un poco en vez del momento del día representado de 0..23 voy a usar sinus y cosinus ya que:

* Captura la naturaleza cíclica de las horas del día.
* Evita problemas con el modelo interpretando incorrectamente que 23 y 0 están lejos entre sí.
* Mejora el aprendizaje en modelos basados en redes neuronales y algoritmos sensibles a la distancia.

In [19]:
def agregar_hour_of_day_ciclico(df, datetime_col=None):
    """
    Agrega columnas 'hour_sin' y 'hour_cos' que representan la hora de manera cíclica.
    
    Param:
    ------
    df : pd.DataFrame
        Debe tener un índice datetime o una columna datetime.
    datetime_col : str or None
        Si None, se asume que el df.index es datetime.
        Si no, se utiliza la columna dada.
    
    Returns:
    --------
    df_out : pd.DataFrame
        Mismo DataFrame con columnas 'hour_sin' y 'hour_cos'.
    """
    df_out = df.copy()
    
    if datetime_col is None:
        # Asume df.index es datetime
        hours = df_out.index.hour
    else:
        # Extrae de la columna
        hours = pd.to_datetime(df_out[datetime_col]).dt.hour
    
    # Codificación cíclica
    df_out['hour_sin'] = np.sin(2 * np.pi * hours / 24)
    df_out['hour_cos'] = np.cos(2 * np.pi * hours / 24)
    
    return df_out

In [20]:
df_color = agregar_hour_of_day_ciclico(df_color)

### Incorporar la duración de la parada anterior

Como veremos más adelante, suprimiré los datos de la máquina cuando esta se encuentre en estado "parada" ya que en ese momento el color es nulo e irrelevante. Para que de cierta manera el modelo sepa que ha habido un "paro" lo que haré es en la fila siguiente al último estado de parada le añadiré la **duración de la parada**, así ya podré eliminar esas filas sin perder la información. 

Por tanto, antes de eliminar las filas estado='parada', recorremos el dataframe y, cada vez que detectamos un bloque de paradas (una o varias filas consecutivas), calculamos cuánto duró. En la fila inmediatamente posterior (la primera fila donde estado != 'parada' tras ese bloque) guardamos un número en la columna "duracion_parada_anterior" (en minutos, o intervalos, etc.). Si no hubo parada recientemente, esa columna se pone a 0 (o NaN).

In [21]:
def add_duracion_parada_anterior(df):
    """
    Recorre el DataFrame en orden cronológico y, por cada bloque de estado='parada',
    calcula la duración total. En la fila inmediatamente siguiente (primera no-parada)
    se añade esa duración a una columna 'duracion_parada_anterior' (en minutos).
    
    Asume que el index del df es un DatetimeIndex o que, al menos,
    df esté ordenado cronológicamente.
    """
    df = df.sort_index().copy()
    df['duracion_parada_anterior'] = 0.0  # de inicio
    
    n = len(df)
    if n == 0:
        return df
    
    parada_start_idx = None
    
    for i in range(n):
        row_estado = df['estado'].iloc[i]
        
        if row_estado == 'parada':
            # Si la parada_start_idx es None, la iniciamos
            if parada_start_idx is None:
                parada_start_idx = i
        else:
            # Si estamos en un estado != parada
            if parada_start_idx is not None:
                # Significa que acabamos de salir de un bloque de parada
                # Calculamos la duración
                start_time = df.index[parada_start_idx]
                end_time   = df.index[i]  # La fila actual (la 1ª no-parada)
                
                # En minutos
                duracion_parada = (end_time - start_time).total_seconds() / 60.0
                
                # Guardamos en la fila actual
                df.loc[df.index[i], 'duracion_parada_anterior'] = duracion_parada
                
                parada_start_idx = None  # Reseteamos
                
            else:
                # No venimos de una parada
                df.loc[df.index[i], 'duracion_parada_anterior'] = 0.0
    
    return df


In [22]:
df_color = add_duracion_parada_anterior(df_color)

### Incorporar el tiempo en cada estado

Aunque ya tenemos las columnas de estado, eso no nos indica cuánto tiempo consecutivo está en cada estado, este será un contador que se reseterá cada vez que el estado cambie.

Esto nos ayudará a capturar:

* Derivas acumuladas.
* Efecto de transiciones recientes vs. largas permanencias.
* Diferencias de comportamiento al rebasar cierto umbral de tiempo en el mismo estado.

In [23]:
def calcular_tiempo_en_estado(df, col_estado='estado'):
    """
    Crea una columna 'tiempo_en_estado' que cuenta 
    cuántas filas consecutivas lleva la máquina en el mismo estado
    (puesta, normal o alta). 
    Se resetea a 0 cuando cambia el estado.
    
    Param:
    ------
    df : pd.DataFrame
        Debe estar ordenado cronológicamente
        y contener una columna 'estado' con valores en {parada, puesta, normal, alta}.
    col_estado : str
        Nombre de la columna del estado.

    Returns:
    --------
    df_out : pd.DataFrame
        Mismo DF con una columna nueva 'tiempo_en_estado' (int).
    """
    df_out = df.copy()
    df_out['tiempo_en_estado'] = 0
    
    if len(df_out) == 0:
        return df_out
    
    # Iniciamos un contador
    contador = 0
    # Asignar para la primera fila
    df_out.loc[df_out.index[0], 'tiempo_en_estado'] = 0
    estado_anterior = df_out[col_estado].iloc[0]

    for i in range(1, len(df_out)):
        estado_actual = df_out[col_estado].iloc[i]
        if estado_actual == estado_anterior:
            # mismo estado => incrementamos
            contador += 1
        else:
            # distinto estado => reset
            contador = 0
        
        df_out.loc[df_out.index[i], 'tiempo_en_estado'] = contador
        estado_anterior = estado_actual

    return df_out


In [24]:
df_color = calcular_tiempo_en_estado(df_color)
print(df_color.head())
print(df_color.tail())


                         tmin  tmed  tmax  horatmin  horatmax  hrMin  hrMedia  \
datetime                                                                        
2023-01-02 00:00:00.000   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:02:52.800   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:05:45.600   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:08:38.400   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   
2023-01-02 00:11:31.200   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   

                         hrMax horaHrMin horaHrMax  ...   A  A_base   B  \
datetime                                            ...                   
2023-01-02 00:00:00.000   93.0  00:00:00  23:59:00  ... NaN     NaN NaN   
2023-01-02 00:02:52.800   93.0  00:00:00  23:59:00  ... NaN     NaN NaN   
2023-01-02 00:05:45.600   93.0  00:00:00  23:59:00  ... NaN     NaN NaN   
2023-01-02 00:08:38.400   93.0  00:00:00  23:59:00  ... N

## Guardar fichero con todos los datos

Este dataframe contiene todos los datos iniciales y lo vamos a guardar en un fichero ya que luego nos ayudará a reconstruir el dataframe inicial agregándole las predicciones

In [25]:
with open(FIC_TOTAL, 'wb') as f:
    pickle.dump(df_color, f)

## Suprimir filas con estado de *parada* e incorporar *seq_time*

Como los estados de parada no llevan color y en el caso que nos ocupa nos interesa la predicción del color, las columnas con el estado parada las suprimiré. Este hecho provocará un salto en el tiempo que puede hacer que los modelos que esperan datos secuenciales, pueden no ser óptimos. Por eso, incorporamos la columna *seq_time* que lo que hace es, una vez borrados las filas que no nos interesan, crean una secuencia consecutiva.

Más adelante, tendremos que recuperar las filas borradas para representar los resultados.

In [26]:
def filtrar_y_renumerar(df, estado_parada='parada'):
    """
    Elimina filas donde estado='parada',
    crea un 'seq_time' secuencial para la RNN (0,1,2,...).
    """
    df_filtered = df[df['estado'] != estado_parada].copy()
    
    # Creamos un nuevo índice secuencial (orden cronológico)
    df_filtered = df_filtered.sort_index()
    df_filtered['seq_time'] = np.arange(len(df_filtered))
    
    return df_filtered

In [27]:
df_sin_parada = filtrar_y_renumerar(df_color)

## Guardamos los datos sin estado "parada"

Ahora que ya tengo datos para procesar, voy a guardarlo en disco para poder usarlo luego.

In [28]:
os.chdir(RUTA_DATOS)

# Guardar un objeto en un fichero
with open(FIC_SALIDA, 'wb') as f:
    pickle.dump(df_sin_parada, f)

df_color.dtypes
df_sin_parada

,tmin,tmed,tmax,horatmin,horatmax,hrMin,hrMedia,hrMax,horaHrMin,horaHrMax,...,A_base,B,B_base,Delta_E,patron_id,hour_sin,hour_cos,duracion_parada_anterior,tiempo_en_estado,seq_time
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:31:40.800,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-12.960367,5.271659,5.271659,0.000000,1,0.000000,1.000000,31.68,0,0
2023-01-02 00:34:33.600,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-12.960367,5.271659,5.271659,0.000000,1,0.000000,1.000000,0.00,1,1
2023-01-02 00:37:26.400,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-12.960367,5.271659,5.271659,0.000000,1,0.000000,1.000000,0.00,2,2
2023-01-02 00:40:19.200,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-12.960367,5.271659,5.271659,0.000000,1,0.000000,1.000000,0.00,3,3
2023-01-02 00:43:12.000,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-12.960367,5.271659,5.271659,0.000000,1,0.000000,1.000000,0.00,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-01 23:45:36.000,9.3,14.6,19.9,05:00:00,17:40:00,49.0,55.0,93.0,17:40:00,00:40:00,...,23.146777,-36.263861,-26.593861,23.686566,104,-0.258819,0.965926,0.00,967,270672
2024-07-01 23:48:28.800,9.3,14.6,19.9,05:00:00,17:40:00,49.0,55.0,93.0,17:40:00,00:40:00,...,23.146777,-36.273861,-26.593861,23.711061,104,-0.258819,0.965926,0.00,968,270673
2024-07-01 23:51:21.600,9.3,14.6,19.9,05:00:00,17:40:00,49.0,55.0,93.0,17:40:00,00:40:00,...,23.146777,-36.283861,-26.593861,23.735556,104,-0.258819,0.965926,0.00,969,270674


In [29]:
print(df_sin_parada.groupby('hour_sin')['hour_sin'].sum())
print(df_sin_parada.groupby('estado')['tiempo_en_estado'].count())

hour_sin
-1.000000e+00   -1.148700e+04
-9.659258e-01   -1.109559e+04
-9.659258e-01   -1.056723e+04
-8.660254e-01   -9.948034e+03
-8.660254e-01   -9.948034e+03
-7.071068e-01   -8.122536e+03
-7.071068e-01   -8.122536e+03
-5.000000e-01   -5.743500e+03
-5.000000e-01   -5.743500e+03
-2.588190e-01   -2.831480e+03
-2.588190e-01   -2.973054e+03
 0.000000e+00    0.000000e+00
 1.224647e-16    1.406752e-12
 2.588190e-01    2.973054e+03
 2.588190e-01    2.831480e+03
 5.000000e-01    1.148700e+04
 7.071068e-01    1.624507e+04
 8.660254e-01    9.948034e+03
 8.660254e-01    9.948034e+03
 9.659258e-01    2.166282e+04
 1.000000e+00    1.148700e+04
Name: hour_sin, dtype: float64
estado
alta       84729
normal    183289
puesta      2659
Name: tiempo_en_estado, dtype: int64
